In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import log_loss
from pandas.tools.plotting import scatter_matrix
from scipy.optimize import minimize
from project import utils

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

pd.options.mode.chained_assignment = None

In [ ]:
X, X_test, Y = utils.load_ensemble_data(recipe=[
    '01_XGB_split',
     '11_LGB_split',
    '09_NN_extended',
    '13_XGB_flip2'    
])

In [ ]:
from scipy.stats.mstats import gmean
print(log_loss(Y, gmean(X,axis=1)))
print(log_loss(Y, X.mean(axis=1)))

In [ ]:
def blended(weight, X):
    return np.sum(X * weight, axis=1)

In [ ]:
def error(weights):
    return log_loss(Y, blended(weights, X))

In [ ]:
starting_values = [0.5] * X.shape[1]
cons = ({'type': 'eq', 'fun': lambda w: 1-sum(w)})
bounds = [(0, 1)] * X.shape[1]

In [ ]:
res = minimize(error, starting_values, method='SLSQP', bounds=bounds)

print('Ensemble Score: {best_score}'.format(best_score=res['fun']))
print('Best Weights: {weights}'.format(weights=res['x']))

In [ ]:
result = blended(res['x'], X_test)
result.to_csv('blended.csv', index=False, header=None)

In [ ]:
result = X_test.mean(axis=1)
result.to_csv('averaged.csv', index=False, header=None)

In [ ]:
y1 = pd.read_csv('averaged.csv', header=None).values.ravel()
y2 = pd.read_csv('mixin_solution.csv', header=None).values.ravel()
res = pd.DataFrame(data=(y1 + y2) / 2.0)
res.to_csv('submit-model-merged.csv', index=False, header=False)